In [7]:
## get RNA sequence fasta files and separate RNA monomer from the complex for network features calculation
import pandas as pd
import numpy as np
import glob
import re
import os
import sys
import re
import math
import cmath

rna_seq = {}
f_g = glob.glob('pdb/*.pdb')
for f_i in f_g:
    rnaname = os.path.basename(f_i)[:4]
    aanub = 100000
    seq = ''
    with open("pdb/%s.pdb"%(rnaname),"r") as f_r:
        lines = f_r.readlines()
        for line in lines:
            if line[0:4] == "ATOM":
                if aanub != line[22:26]:
                    seq = seq+ line[19:20]
                aanub = line[22:26]
                fw1 = open('graph_pdb/%s.pdb'%rnaname, 'a+') 
                fw1.write(line)
                fw2 = open('fasta/%s.fasta'%rnaname, 'w')
                fw2.write('>'+rnaname+'\n')
                fw2.write(seq)

In [9]:
#######RNA features integration                        ---- written by kaili
import pickle
import glob
import os
from pathlib import Path

nucle_type ={'A':[1,0,0,0],
            'U':[0,1,0,0],
            'C':[0,0,1,0],
            'G':[0,0,0,1]
}
mass ={'A':507.18,
       'U':484.141,
       'C':483.156,
       'G':523.18   
}
PKa ={'A':3.5,
      'U':9.2,
      'C':4.2,
      'G':10.8  
}
rna_features=[]
nucle_4=[]
mass_normal=[]
pka_normal=[]
clon=[]
deg=[]
conserv_normal=[]
asa_normal=[]
f_cont = glob.glob("./pdb/*.pdb")
for fi in f_cont:
    rnaname = os.path.basename(fi)[:4]
    nucle_list=[]
    mass_list=[]
    pka_list=[]
    with open(r"./fasta/%s.fasta"%(rnaname),"r") as f1:
        for line in f1:
            if line[0] != ">":
                seq = line.strip()
                for nucle in seq:
                    nucle_onehot=nucle_type[nucle]
                    ma=float(mass[nucle])/523.18
                    pk=float(PKa[nucle])/10.8
                    nucle_list.append(nucle_onehot)
                    mass_list.append(ma)
                    pka_list.append(pk)
                nucle_4.append(nucle_list)
                mass_normal.append(mass_list)
                pka_normal.append(pka_list)
    with open(r'./features/closeness/Outputs/%s_closeness.txt'%(rnaname), 'r') as f2:
        cl_list=[]
        for line in f2:
            cl = line.strip()
            cl_list.append(float(cl))      
        clon.append(cl_list)
    with open(r'./features/degree/Outputs/%s_degree.txt'%(rnaname), 'r') as f3:
        for line in f3:
            de = line[:-1].strip().split(',')
        deg.append(de)    
    with open(r'./features/conservation/%s/consurf.grades'%(rnaname), 'r') as f4:
        conser_list=[]
        for _ in range(15):
            next(f4)
        for line in f4:
            if line[31:32]== '':
                break
            else:
                conser =line[31:32]
                conser_list.append(float(conser)/9)
        conserv_normal.append(conser_list)
    with open(r'./features/ASA/%s.txt'%(rnaname), 'r') as f5:
#         print(rna_name)
        asa_list=[]
        for _ in range(1):
            next(f5)
        for line in f5:
            asa_value = line.strip().split('	')[2]
            asa_list.append(float(asa_value)/400)  # maximum value of all the data
        asa_normal.append(asa_list)       
data_pack = []
for i in range(0,1): # files traverse
    data_pack.append([])
    for j in range(0,len(nucle_4[i])):
        temp = []
        temp += nucle_4[i][j]
        temp.append(clon[i][j])
        temp.append(int(deg[i][j])/24.0)  # maximum value of all the data
        temp.append(conserv_normal[i][j])
        temp.append(asa_normal[i][j])
        temp.append(mass_normal[i][j])
        temp.append(pka_normal[i][j])
        data_pack[i].append(temp)
pickle.dump(data_pack,Path('./features/integration/demo.pkl').open('wb'))

In [10]:
#######RNA motif features integration
import pickle
import glob
import os
from pathlib import Path
import numpy as np

nucle_type ={'A':[1,0,0,0],
            'U':[0,1,0,0],
            'C':[0,0,1,0],
            'G':[0,0,0,1]
}
mass ={'A':507.18,
       'U':484.141,
       'C':483.156,
       'G':523.18   
}
PKa ={'A':3.5,
      'U':9.2,
      'C':4.2,
      'G':10.8  
}
rna_features=[]
nucle_4=[]
mass_normal=[]
pka_normal=[]
clon=[]
deg=[]
conserv_normal=[]
asa_normal=[]
f_cont = glob.glob("./pdb/*.pdb")
for fi in f_cont:
    rnaname = os.path.basename(fi)[:4]
    nucle_list=[]
    mass_list=[]
    pka_list=[]
    with open(r"./fasta/%s.fasta"%(rnaname),"r") as f1:
        for line in f1:
            if line[0] != ">":
                seq = line.strip()
                for nucle in seq:
                    nucle_onehot=nucle_type[nucle]
                    ma=float(mass[nucle])/523.18
                    pk=float(PKa[nucle])/10.8
                    nucle_list.append(nucle_onehot)
                    mass_list.append(ma)
                    pka_list.append(pk)
                nucle_4.append(nucle_list)
                mass_normal.append(mass_list)
                pka_normal.append(pka_list)
    with open(r'./features/closeness/Outputs/%s_closeness.txt'%(rnaname), 'r') as f2:
        cl_list=[]
        for line in f2:
            cl = line.strip()
            cl_list.append(float(cl))      
        clon.append(cl_list)
    with open(r'./features/degree/Outputs/%s_degree.txt'%(rnaname), 'r') as f3:
        for line in f3:
            de = line[:-1].strip().split(',')
        deg.append(de)    
    with open(r'./features/conservation/%s/consurf.grades'%(rnaname), 'r') as f4:
        conser_list=[]
        for _ in range(15):
            next(f4)
        for line in f4:
            if line[31:32]== '':
                break
            else:
                conser =line[31:32]
                conser_list.append(float(conser)/9)
        conserv_normal.append(conser_list)
    with open(r'./features/ASA/%s.txt'%(rnaname), 'r') as f5:
        asa_list=[]
        for _ in range(1):
            next(f5)
        for line in f5:
            asa_value = line.strip().split('	')[2]
            asa_list.append(float(asa_value)/400)  # maximum value of all the data
        asa_normal.append(asa_list)       
data_pack = []
for i in range(0,1):   # files traverse
    data_pack.append([])
    for j in range(0,len(nucle_4[i])):
        temp = []
        temp += nucle_4[i][j]
        temp.append(clon[i][j])
        temp.append(int(deg[i][j])/24.0)  # maximum value of all the data
        temp.append(conserv_normal[i][j])
        temp.append(asa_normal[i][j])
        temp.append(mass_normal[i][j])
        temp.append(pka_normal[i][j])
        data_pack[i].append(temp)
a = []
for m in range(0,1):
    i=0
    for n in range(0,len(data_pack[m])):
        le=10
        lis=[0]*le
        motif_list=[]
        if n-1 <0:
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(data_pack[m][n])
            motif_list.append(data_pack[m][n+1])
            motif_list.append(data_pack[m][n+2])
            motif_list.append(data_pack[m][n+3])
            motif_list.append(data_pack[m][n+4])
            motif_list.append(data_pack[m][n+5])
        elif n-1>=0 and n-2 <0:
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(data_pack[m][n-1])
            motif_list.append(data_pack[m][n])
            motif_list.append(data_pack[m][n+1])
            motif_list.append(data_pack[m][n+2])
            motif_list.append(data_pack[m][n+3])
            motif_list.append(data_pack[m][n+4])
            motif_list.append(data_pack[m][n+5])
        elif n-1>=0 and n-2>=0 and n-3 <0:
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(data_pack[m][n-2])
            motif_list.append(data_pack[m][n-1])
            motif_list.append(data_pack[m][n])
            motif_list.append(data_pack[m][n+1])
            motif_list.append(data_pack[m][n+2])
            motif_list.append(data_pack[m][n+3])
            motif_list.append(data_pack[m][n+4])
            motif_list.append(data_pack[m][n+5])
        elif n-1>=0 and n-2>=0 and n-3 >=0 and n-4 <0:
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(data_pack[m][n-3])
            motif_list.append(data_pack[m][n-2])
            motif_list.append(data_pack[m][n-1])
            motif_list.append(data_pack[m][n])
            motif_list.append(data_pack[m][n+1])
            motif_list.append(data_pack[m][n+2])
            motif_list.append(data_pack[m][n+3])
            motif_list.append(data_pack[m][n+4])
            motif_list.append(data_pack[m][n+5])
        elif n-1>=0 and n-2>=0 and n-3 >=0 and n-4 >=0 and n-5<0:
            motif_list.append(lis)
            motif_list.append(data_pack[m][n-4])
            motif_list.append(data_pack[m][n-3])
            motif_list.append(data_pack[m][n-2])
            motif_list.append(data_pack[m][n-1])
            motif_list.append(data_pack[m][n])
            motif_list.append(data_pack[m][n+1])
            motif_list.append(data_pack[m][n+2])
            motif_list.append(data_pack[m][n+3])
            motif_list.append(data_pack[m][n+4])
            motif_list.append(data_pack[m][n+5])
        elif n+1 == len(data_pack[m]):
            motif_list.append(data_pack[m][n-5])
            motif_list.append(data_pack[m][n-4])
            motif_list.append(data_pack[m][n-3])
            motif_list.append(data_pack[m][n-2])
            motif_list.append(data_pack[m][n-1])
            motif_list.append(data_pack[m][n])
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(lis)
        elif n+1 <= len(data_pack[m]) and n+2 == len(data_pack[m]):
            motif_list.append(data_pack[m][n-5])
            motif_list.append(data_pack[m][n-4])
            motif_list.append(data_pack[m][n-3])
            motif_list.append(data_pack[m][n-2])
            motif_list.append(data_pack[m][n-1])
            motif_list.append(data_pack[m][n])
            motif_list.append(data_pack[m][n+1])
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(lis)
        elif n+2 <= len(data_pack[m]) and n+3 == len(data_pack[m]):
            motif_list.append(data_pack[m][n-5])
            motif_list.append(data_pack[m][n-4])
            motif_list.append(data_pack[m][n-3])
            motif_list.append(data_pack[m][n-2])
            motif_list.append(data_pack[m][n-1])
            motif_list.append(data_pack[m][n])
            motif_list.append(data_pack[m][n+1])
            motif_list.append(data_pack[m][n+2])
            motif_list.append(lis)
            motif_list.append(lis)
            motif_list.append(lis)
        elif n+3 <= len(data_pack[m]) and n+4 == len(data_pack[m]):
            motif_list.append(data_pack[m][n-5])
            motif_list.append(data_pack[m][n-4])
            motif_list.append(data_pack[m][n-3])
            motif_list.append(data_pack[m][n-2])
            motif_list.append(data_pack[m][n-1])
            motif_list.append(data_pack[m][n])
            motif_list.append(data_pack[m][n+1])
            motif_list.append(data_pack[m][n+2])
            motif_list.append(data_pack[m][n+3])
            motif_list.append(lis)
            motif_list.append(lis)
        elif n+4 <= len(data_pack[m]) and n+5 == len(data_pack[m]):
            motif_list.append(data_pack[m][n-5])
            motif_list.append(data_pack[m][n-4])
            motif_list.append(data_pack[m][n-3])
            motif_list.append(data_pack[m][n-2])
            motif_list.append(data_pack[m][n-1])
            motif_list.append(data_pack[m][n])
            motif_list.append(data_pack[m][n+1])
            motif_list.append(data_pack[m][n+2])
            motif_list.append(data_pack[m][n+3])
            motif_list.append(data_pack[m][n+4])
            motif_list.append(lis)
        else:
            motif_list.append(data_pack[m][n-5])
            motif_list.append(data_pack[m][n-4])
            motif_list.append(data_pack[m][n-3])
            motif_list.append(data_pack[m][n-2])
            motif_list.append(data_pack[m][n-1])
            motif_list.append(data_pack[m][n])
            motif_list.append(data_pack[m][n+1])
            motif_list.append(data_pack[m][n+2])
            motif_list.append(data_pack[m][n+3])
            motif_list.append(data_pack[m][n+4])
            motif_list.append(data_pack[m][n+5])
        a.append(np.array(motif_list))
pickle.dump(a,Path('./features/integration/demo_mot11.pkl').open('wb'))

In [11]:
### get RNA index file
import os
import csv
import shutil
import pandas as pd
import pickle
from pathlib import Path

pdbid = []
seq_indx = []
len_list = []
seq_nums_list = []
name_list = []
length_list = []
idx_list = []
idx = -1
na = -1
seq_nums = -1
f_cont = glob.glob("./fasta/*.fasta")
for fi in f_cont:
    rnaname = os.path.basename(fi)[:4]
    pdbid.append(rnaname)
    with open(r"./fasta/%s.fasta"%(rnaname),"r") as f1:
        for line in f1:
            if line[0] != ">":
                seq = line.strip()
                len_list.append(str(len(seq)))
for i in len_list:
    na += 1
    nucleo_id = -1
    seq_nums += 1
    for j in range(int(i)):
        temp= []
        idx += 1
        nucleo_id += 1
        idx_list.append(idx)
        seq_numbs = seq_nums
        name = pdbid[na]
        length = len_list[na]
        seq_nums_list.append(seq_numbs)
        name_list.append(name)
        length_list.append(length) 
        temp.append(idx)
        temp.append(nucleo_id)
        temp.append(seq_numbs)
        temp.append(name)
        temp.append(length)
        seq_indx.append(tuple(temp))
pickle.dump(seq_indx,Path('./features/integration/demo_index_all.pkl').open('wb'))

In [12]:
# the index of each motif
train_val = []
for ii in range(29):  # the number of all the nucleotides
    train_val.append(ii)
pickle.dump(train_val,Path('./features/integration/demo_index.pkl').open('wb'))

In [15]:
alllab = []
f_cont = glob.glob("./features/label/*_labels.csv")
for fi in f_cont:
    rnaname = os.path.basename(fi)[:4]
    siglab = []
    with open(r"./features/label/%s_labels.csv"%(rnaname),"r") as f1:
        for line in f1:
            lab = line.strip().split(",")[1]
            siglab.append(lab)
    alllab.append(siglab)
pickle.dump(alllab,Path('./features/integration/demo_label.pkl').open('wb'))